In [15]:
# common packages
import gym
import random
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
#keras packages
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam

In [17]:
# Environment
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

In [18]:
# Plant the seeds
seed = 123
env.seed(seed)
random.seed(seed)
np.random.seed(seed)

In [19]:
# Parameters used for the model
discount_factor = 0.99
epsilon_decay = 0.995
epsilon_min = 0.01
batch_size = 32
train_start = 1000
memory_size = 10000
n_episodes = 2000
n_win_ticks = 195
n_avg_scores = 100
render = False

In [28]:
# Building the model
def build_model():
    model = Sequential()
    model.add(Dense(96, input_dim=state_size, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(48, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(24, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(action_size, kernel_initializer='he_uniform', activation='relu'))
    model.compile(Adam(lr=0.001), loss='mse')
    return model

In [29]:
model = build_model()
target_model = build_model()

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 96)                480       
_________________________________________________________________
dense_4 (Dense)              (None, 48)                4656      
_________________________________________________________________
dense_5 (Dense)              (None, 24)                1176      
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 50        
Total params: 6,362
Trainable params: 6,362
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Training helpers
# Source: https://github.com/yanpanlau/CartPole/blob/master/DQN/CartPole_DQN.py
def update_target_model():
    target_model.set_weights(model.get_weights())

def get_action(state, epsilon):
    return np.random.randint(action_size) if np.random.rand() <= epsilon else np.argmax(model.predict(state)[0])

def train_replay():
    if len(memory) < train_start:
        return
    minibatch = random.sample(memory,  min(batch_size, len(memory)))
    state_t, action_t, reward_t, state_t1, terminal = zip(*minibatch)
    state_t = np.concatenate(state_t)
    state_t1 = np.concatenate(state_t1)
    targets = model.predict(state_t)
    Q_sa = target_model.predict(state_t1)
    targets[range(batch_size), action_t] = reward_t + discount_factor * np.max(Q_sa, axis=1) * np.invert(terminal)
    model.train_on_batch(state_t, targets)

In [31]:

# Training
avg_scores = []
all_scores = []
scores = deque(maxlen=n_avg_scores)
memory = deque(maxlen=memory_size)

def learn_to_balance():
    epsilon = 1.0 # Start with randomness
    has_won = False

    for e in range(n_episodes):
        done = False
        score = 0
        state = env.reset()
        state = np.reshape(state, [1, state_size])

        while not done:
            action = get_action(state, epsilon)
            next_state, reward, done, info = env.step(action)
            next_state = np.reshape(next_state, [1, state_size])

            memory.append((state, action, reward if not done else -100, next_state, done))
            if epsilon > epsilon_min:
                epsilon *= epsilon_decay # Decrease randomness
            train_replay()
            score += reward
            state = next_state
            
            if render:
                env.render()

            if done:
                env.reset()
                update_target_model()
                scores.append(score)
                all_scores.append(score)
                avg_score = np.mean(scores)
                avg_scores.append(avg_score)

                if not has_won and e >= n_avg_scores and avg_score >= n_win_ticks:
                    # Find first score greater than 195 where the average is >= 195 over the following 100 episodes.
                    solution_episode_idx = max(next(x[0] for x in enumerate(all_scores) if x[1] >= n_win_ticks), e - n_win_ticks) 
                    print('Solved after {} tries! \o/'.format(solution_episode_idx))
                    has_won = True

                if e % n_avg_scores == 0:
                    print('[Episode {}] Average Score: {}'.format(e, avg_score))

In [ ]:

# Train model
# The challenge is solved at episode 8. The mean score remains >= 195 for the following 100 episodes.
# A higher score is very likely achievable through hyperparameter optimization and seed exploration.
# A better performance metric would span more episodes and cover a range of initial environment seeds.
learn_to_balance()

# Plot results
plt.figure(figsize=(12,6))
plt.plot(all_scores, color='blue')
plt.plot(avg_scores, color='red')
plt.show()

[Episode 0] Average Score: 14.0
Instructions for updating:
Use tf.cast instead.
[Episode 100] Average Score: 168.31
Solved after 42 tries! \o/
[Episode 200] Average Score: 322.44
